# Working Notebook

__Phase 3 Project - Chicago Traffic Crash Classification__

__Business Understanding__

__Data Understanding and Preparation__

Load dependencies

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Import data files

In [2]:
crashes = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_Crashes.csv')
people = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_People.csv')
vehicles = pd.read_csv('Chicago-Crashes/data/Traffic_Crashes_-_Vehicles.csv')

/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (12,13,20,21,22,23,24,25,26,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE_EST_I               41029 non-null   object 
 3   CRASH_DATE                     541142 non-null  object 
 4   POSTED_SPEED_LIMIT             541142 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 6   DEVICE_CONDITION               541142 non-null  object 
 7   WEATHER_CONDITION              541142 non-null  object 
 8   LIGHTING_CONDITION             541142 non-null  object 
 9   FIRST_CRASH_TYPE               541142 non-null  object 
 10  TRAFFICWAY_TYPE                541142 non-null  object 
 11  LANE_CNT                       198968 non-null  float64
 12  ALIGNMENT                     

In [4]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

In [5]:
percent_missing = people.isnull().sum() * 100 / len(people)
missing_value_df = pd.DataFrame({'column_name': people.columns,
                                 'percent_missing': percent_missing})

pd.set_option('display.max_rows', None)
missing_value_df

,column_name,percent_missing
PERSON_ID,PERSON_ID,0.000000
PERSON_TYPE,PERSON_TYPE,0.000000
CRASH_RECORD_ID,CRASH_RECORD_ID,0.000000
RD_NO,RD_NO,0.847504
VEHICLE_ID,VEHICLE_ID,1.973745
CRASH_DATE,CRASH_DATE,0.000000
SEAT_NO,SEAT_NO,79.551527
CITY,CITY,26.428584
STATE,STATE,25.553733
ZIPCODE,ZIPCODE,32.829269


In [6]:
percent_missing = vehicles.isnull().sum() * 100 / len(vehicles)
missing_value_df = pd.DataFrame({'column_name': vehicles.columns,
                                 'percent_missing': percent_missing})

pd.set_option('display.max_rows', None)
missing_value_df

,column_name,percent_missing
CRASH_UNIT_ID,CRASH_UNIT_ID,0.000000
CRASH_RECORD_ID,CRASH_RECORD_ID,0.000000
RD_NO,RD_NO,0.856252
CRASH_DATE,CRASH_DATE,0.000000
UNIT_NO,UNIT_NO,0.000000
UNIT_TYPE,UNIT_TYPE,0.144229
NUM_PASSENGERS,NUM_PASSENGERS,85.015358
VEHICLE_ID,VEHICLE_ID,2.287885
CMRC_VEH_I,CMRC_VEH_I,98.150853
MAKE,MAKE,2.288337


In [7]:
percent_missing = crashes.isnull().sum() * 100 / len(crashes)
missing_value_df = pd.DataFrame({'column_name': crashes.columns,
                                 'percent_missing': percent_missing})

pd.set_option('display.max_rows', None)
missing_value_df

,column_name,percent_missing
CRASH_RECORD_ID,CRASH_RECORD_ID,0.000000
RD_NO,RD_NO,0.848576
CRASH_DATE_EST_I,CRASH_DATE_EST_I,92.418071
CRASH_DATE,CRASH_DATE,0.000000
POSTED_SPEED_LIMIT,POSTED_SPEED_LIMIT,0.000000
TRAFFIC_CONTROL_DEVICE,TRAFFIC_CONTROL_DEVICE,0.000000
DEVICE_CONDITION,DEVICE_CONDITION,0.000000
WEATHER_CONDITION,WEATHER_CONDITION,0.000000
LIGHTING_CONDITION,LIGHTING_CONDITION,0.000000
FIRST_CRASH_TYPE,FIRST_CRASH_TYPE,0.000000


In [11]:
perc = 80.0 # Like N %
min_count =  int(((100-perc)/100)*crashes.shape[0] + 1)
crashes_mod = crashes.dropna(axis=1, 
                            thresh=min_count)

In [18]:
perc = 80.0 # Like N %
min_count =  int(((100-perc)/100)*people.shape[0] + 1)
people_mod = people.dropna(axis=1, 
                            thresh=min_count)

In [12]:
crashes_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE                     541142 non-null  object 
 3   POSTED_SPEED_LIMIT             541142 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 5   DEVICE_CONDITION               541142 non-null  object 
 6   WEATHER_CONDITION              541142 non-null  object 
 7   LIGHTING_CONDITION             541142 non-null  object 
 8   FIRST_CRASH_TYPE               541142 non-null  object 
 9   TRAFFICWAY_TYPE                541142 non-null  object 
 10  LANE_CNT                       198968 non-null  float64
 11  ALIGNMENT                      541142 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [20]:
people_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

In [ ]:
people_mod.drop([''])

In [21]:
people_mod.shape

(1195747, 22)

In [22]:
perc = 80.0 # Like N %
min_count =  int(((100-perc)/100)*vehicles.shape[0] + 1)
vehicles_mod = vehicles.dropna(axis=1, 
                            thresh=min_count)

In [23]:
vehicles_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107267 entries, 0 to 1107266
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   CRASH_UNIT_ID        1107267 non-null  int64  
 1   CRASH_RECORD_ID      1107267 non-null  object 
 2   RD_NO                1097786 non-null  object 
 3   CRASH_DATE           1107267 non-null  object 
 4   UNIT_NO              1107267 non-null  int64  
 5   UNIT_TYPE            1105670 non-null  object 
 6   VEHICLE_ID           1081934 non-null  float64
 7   MAKE                 1081929 non-null  object 
 8   MODEL                1081786 non-null  object 
 9   LIC_PLATE_STATE      987487 non-null   object 
 10  VEHICLE_YEAR         905696 non-null   float64
 11  VEHICLE_DEFECT       1081934 non-null  object 
 12  VEHICLE_TYPE         1081934 non-null  object 
 13  VEHICLE_USE          1081934 non-null  object 
 14  TRAVEL_DIRECTION     1081934 non-null  object 
 15

In [31]:
vehicles_mod.drop(['RD_NO', 'CRASH_DATE', 'UNIT_NO', 'AREA_01_I'], axis =1, inplace=True)

/Users/meaghanross/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [32]:
vehicles_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107267 entries, 0 to 1107266
Data columns (total 15 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   CRASH_UNIT_ID        1107267 non-null  int64  
 1   CRASH_RECORD_ID      1107267 non-null  object 
 2   UNIT_TYPE            1105670 non-null  object 
 3   VEHICLE_ID           1081934 non-null  float64
 4   MAKE                 1081929 non-null  object 
 5   MODEL                1081786 non-null  object 
 6   LIC_PLATE_STATE      987487 non-null   object 
 7   VEHICLE_YEAR         905696 non-null   float64
 8   VEHICLE_DEFECT       1081934 non-null  object 
 9   VEHICLE_TYPE         1081934 non-null  object 
 10  VEHICLE_USE          1081934 non-null  object 
 11  TRAVEL_DIRECTION     1081934 non-null  object 
 12  MANEUVER             1081934 non-null  object 
 13  OCCUPANT_CNT         1081934 non-null  float64
 14  FIRST_CONTACT_POINT  1073691 non-null  object 
dty

In [33]:
crashes_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541142 entries, 0 to 541141
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                541142 non-null  object 
 1   RD_NO                          536550 non-null  object 
 2   CRASH_DATE                     541142 non-null  object 
 3   POSTED_SPEED_LIMIT             541142 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         541142 non-null  object 
 5   DEVICE_CONDITION               541142 non-null  object 
 6   WEATHER_CONDITION              541142 non-null  object 
 7   LIGHTING_CONDITION             541142 non-null  object 
 8   FIRST_CRASH_TYPE               541142 non-null  object 
 9   TRAFFICWAY_TYPE                541142 non-null  object 
 10  LANE_CNT                       198968 non-null  float64
 11  ALIGNMENT                      541142 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [34]:
crashes_mod.drop(['RD_NO','REPORT_TYPE', 'INTERSECTION_RELATED_I', 'DATE_POLICE_NOTIFIED', 'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME'], axis =1, inplace=True)

In [35]:
people_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1195747 entries, 0 to 1195746
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1195747 non-null  object 
 1   PERSON_TYPE            1195747 non-null  object 
 2   CRASH_RECORD_ID        1195747 non-null  object 
 3   RD_NO                  1185613 non-null  object 
 4   VEHICLE_ID             1172146 non-null  float64
 5   CRASH_DATE             1195747 non-null  object 
 6   SEAT_NO                244512 non-null   float64
 7   CITY                   879728 non-null   object 
 8   STATE                  890189 non-null   object 
 9   ZIPCODE                803192 non-null   object 
 10  SEX                    1177665 non-null  object 
 11  AGE                    852450 non-null   float64
 12  DRIVERS_LICENSE_STATE  705542 non-null   object 
 13  DRIVERS_LICENSE_CLASS  608027 non-null   object 
 14  SAFETY_EQUIPMENT  

In [37]:
people_mod.drop(['CRASH_DATE', 'CITY', 'ZIPCODE','RD_NO'], axis =1, inplace=True)